This notebook shows different visualizations of the algorithms performances

In [ ]:
import config

In [ ]:
import os
import pickle
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Open all the score

In [ ]:
config.result

In [ ]:
date = ''

In [ ]:
path = os.path.join(config.result, date)
scores = {}
for dname in os.listdir(path):
    unpick = pickle.load(open(os.path.join(path, dname), 'rb'))
    openScore = {(category, method, metric): unpick["Score"][method][category][metric] 
                    for method in unpick["Score"] 
                    for category in unpick["Score"][method]
                    for metric in unpick["Score"][method][category]}
    openScore[("Dataset", "N_Classes")] = unpick["N_Classes"]
    openScore[("Dataset", "Constraint")] = unpick["Constraint"]
    openScore[("Dataset", "Labels")] = unpick["Labels"]
    openScore[("Dataset", "Train")] = unpick["Train"]
    scores[unpick['Name']] = openScore
print("Open {} result files from {}".format(len(scores), date))

In [ ]:
scores = pd.DataFrame.from_dict(scores, orient="index")
scores.head()

# Comparison

In [ ]:
metric = "Adjusted Rand"
category = "test" # 'all', 'train' or 'test'

## Global performance

In [ ]:
measure = scores.loc[:, (category, slice(None), metric)]
measure = measure.T.reset_index(level=[0, 2], drop=True).T
sns.boxplot(data = measure, orient="h")
plt.ylabel("Methods")
plt.xlabel(metric)

In [ ]:
sns.violinplot(data = measure, orient="h")
plt.ylabel("Methods")
plt.xlabel(metric)

## First ranked

Display which method has the best score on every dataset

In [ ]:
# Count rank position
rank = measure.rank(axis = 1, method = 'min',ascending = False).apply(lambda x: x.value_counts()).T 
rank = rank.sort_values(by=[1, 2, 3], ascending = False) # Reorder
rank.plot.barh(stacked=True)
plt.xlabel("Number dataset")
plt.ylabel("Methods")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="Rank")

# Analysis Baysian

In [ ]:
method = "mpckm"
metric = "Adjusted Rand"

## Train vs Test

In [ ]:
measure = scores.loc[:, ([slice(None)], method, metric)]
measure = measure.T.reset_index(level=[1, 2], drop=True).T
measure["Number points"] = scores[("Dataset", "Labels", )].apply(lambda x: len(x.values[0]), axis = 1) 
measure["Number classes"] = scores[("Dataset", "N_Classes", )]
cmap = sns.cubehelix_palette(rot=-.2, as_cmap=True)
plt.title(method)
sns.scatterplot(x="train", y="test", sizes=(10, 200), data=measure, 
                size="Number points", hue="Number classes", palette = cmap, alpha = 0.7)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))